In [86]:
import pandas as pd
import math
import numpy as np

In [87]:
df=pd.read_csv('Data.csv')

In [88]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%Y-%m-%d')
df['InvoiceDate'] = df['InvoiceDate'].dt.strftime('%Y-%m-%d')

In [89]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
944463,575312,22083,PAPER CHAIN KIT RETROSPOT,6,2011-11-09,2.95,13588.0,United Kingdom
944464,575312,23355,HOT WATER BOTTLE KEEP CALM,4,2011-11-09,4.95,13588.0,United Kingdom
944465,575312,22110,BIRD HOUSE HOT WATER BOTTLE,6,2011-11-09,2.55,13588.0,United Kingdom
944466,575312,22037,ROBOT BIRTHDAY CARD,12,2011-11-09,0.42,13588.0,United Kingdom


In [90]:
df2=df[df['InvoiceDate']=='2009-12-01']

In [91]:
df2['Total']=df2['Quantity']*df2['Price']

C:\Users\12967\AppData\Local\Temp\ipykernel_9256\615049507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Total']=df2['Quantity']*df2['Price']


In [92]:
df2

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085.0,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01,2.10,13085.0,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01,1.25,13085.0,United Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
3181,489668,22192,BLUE DINER WALL CLOCK,2,2009-12-01,8.50,16701.0,United Kingdom,17.00
3182,489668,22193,RED DINER WALL CLOCK,4,2009-12-01,8.50,16701.0,United Kingdom,34.00
3183,489668,21479,WHITE SKULL HOT WATER BOTTLE,12,2009-12-01,3.75,16701.0,United Kingdom,45.00
3184,489668,21490,SET OF THREE 50'S GIFT WRAPS,48,2009-12-01,1.65,16701.0,United Kingdom,79.20


In [93]:
df_group=df2.groupby(['Customer ID', 'Country']).agg({'Total': ['sum']}). reset_index()
df_group.columns = df_group.columns.droplevel(1)

In [94]:
df_group

,Customer ID,Country,Total
0,12362.0,Belgium,130.00
1,12490.0,France,603.94
2,12533.0,Germany,929.92
3,12615.0,France,-5.95
4,12636.0,USA,141.00
...,...,...,...
112,17998.0,United Kingdom,229.60
113,18037.0,United Kingdom,21.95
114,18087.0,United Kingdom,394.74
115,18102.0,United Kingdom,2286.24


Интересующее событие - стоимость чека 1 декабря 2009 года,
Событие наблюдение - страна 

In [95]:
df_group['Max_Price']=(df_group['Total']//100+1)*100
df_group['Max_Price'] = df_group['Max_Price'].astype(int) 

In [96]:
country=df_group['Country'].unique()
max_price=df_group['Max_Price'].unique()

In [97]:
df_country=pd.DataFrame(data=country, columns=['country'])
df_country['key']=0

In [98]:
df_max_price=pd.DataFrame(data=max_price, columns=['max_price'])
df_max_price['key']=0

In [99]:
df_prob=df_country.merge(df_max_price, on='key', how='outer')
df_prob['probability']=0

In [100]:
len_df=len(df_group)

In [101]:
def count_row(country, max_price):
    df_temp=df_group[(df_group['Country']==country)&(df_group['Max_Price']==max_price)]
    return len(df_temp)

In [102]:
for i, row in df_prob.iterrows():
    country_temp=row['country']
    max_price_temp=row['max_price']
    res=count_row(country_temp, max_price_temp)
    df_prob.iat[i, 3] = res/len_df

In [107]:
df_prob=df_prob.sort_values('probability', ascending=False)
df_prob

,country,key,max_price,probability
88,United Kingdom,0,0,0.213675
92,United Kingdom,0,300,0.179487
94,United Kingdom,0,400,0.128205
85,United Kingdom,0,200,0.085470
91,United Kingdom,0,600,0.068376
...,...,...,...,...
118,EIRE,0,2300,0.000000
103,EIRE,0,700,0.000000
119,Australia,0,200,0.000000
121,Australia,0,1000,0.000000
